# Imports

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.inspection import permutation_importance
import ta

from tqdm.notebook import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.metrics import f1_score
from imblearn.under_sampling import RandomUnderSampler
pd.set_option('display.float_format', lambda x: '%.3f' % x)
np.set_printoptions(suppress=True)
from pprint import pprint

import gc
import optuna
from sklearn.feature_selection import RFE


/tmp/ipykernel_1250/384002432.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv('Data/Training/pair_features_1_pairs1_300_120.csv')
df.head()

,Ticker_P1,Date,High_P1,Low_P1,Volume_P1,Close_P1,Ticker_P2,High_P2,Low_P2,Volume_P2,Close_P2,abs_spread,abs_spread_mean,abs_spread_std,abs_spread_mean_MA,abs_spread_std_MA,pnls,actual_abs_spread_std
0,LHX,2001-02-08 00:00:00,13.615,13.336,589966.000,8.946,HST,12.697,12.531,289289.000,6.480,2.467,4.442,1.363,2.231,0.320,0.000,0.441
1,LHX,2001-02-09 00:00:00,13.652,12.991,774767.000,8.626,HST,12.717,12.491,368046.000,6.429,2.197,4.441,1.365,2.213,0.311,0.000,0.437
2,LHX,2001-02-12 00:00:00,13.322,12.878,939881.000,8.642,HST,12.736,12.472,436589.000,6.505,2.137,4.439,1.368,2.198,0.306,0.000,0.439
3,LHX,2001-02-13 00:00:00,13.700,13.180,930356.000,8.946,HST,12.814,12.638,435057.000,6.525,2.421,4.437,1.370,2.219,0.306,0.000,0.409
4,LHX,2001-02-14 00:00:00,13.785,13.293,881456.000,8.953,HST,12.717,12.628,560293.000,6.530,2.423,4.436,1.372,2.240,0.305,0.000,0.366


In [3]:
df['pnls'] = df['pnls']*100.0
# df = df[df.pnls != 0]

In [4]:
total_days = df.Date.nunique()
total_days

5739

In [5]:
df.dropna()

,Ticker_P1,Date,High_P1,Low_P1,Volume_P1,Close_P1,Ticker_P2,High_P2,Low_P2,Volume_P2,Close_P2,abs_spread,abs_spread_mean,abs_spread_std,abs_spread_mean_MA,abs_spread_std_MA,pnls,actual_abs_spread_std
0,LHX,2001-02-08 00:00:00,13.615,13.336,589966.000,8.946,HST,12.697,12.531,289289.000,6.480,2.467,4.442,1.363,2.231,0.320,0.000,0.441
1,LHX,2001-02-09 00:00:00,13.652,12.991,774767.000,8.626,HST,12.717,12.491,368046.000,6.429,2.197,4.441,1.365,2.213,0.311,0.000,0.437
2,LHX,2001-02-12 00:00:00,13.322,12.878,939881.000,8.642,HST,12.736,12.472,436589.000,6.505,2.137,4.439,1.368,2.198,0.306,0.000,0.439
3,LHX,2001-02-13 00:00:00,13.700,13.180,930356.000,8.946,HST,12.814,12.638,435057.000,6.525,2.421,4.437,1.370,2.219,0.306,0.000,0.409
4,LHX,2001-02-14 00:00:00,13.785,13.293,881456.000,8.953,HST,12.717,12.628,560293.000,6.530,2.423,4.436,1.372,2.240,0.305,0.000,0.366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,LHX,2023-11-24 00:00:00,191.010,188.050,482300.000,191.010,HST,17.470,17.250,1743300.000,17.031,173.979,179.348,17.309,166.340,4.530,2.333,6.669
5735,LHX,2023-11-27 00:00:00,190.860,187.370,676200.000,187.920,HST,17.490,17.220,4290800.000,17.041,170.879,179.228,17.243,167.126,3.794,2.964,6.320
5736,LHX,2023-11-28 00:00:00,189.740,187.680,726600.000,189.100,HST,17.650,17.320,4449600.000,17.198,171.902,179.111,17.173,167.808,3.313,3.087,5.912
5737,LHX,2023-11-29 00:00:00,189.780,188.060,661800.000,188.660,HST,17.780,17.410,4099200.000,17.051,171.609,178.998,17.111,168.229,3.228,2.687,5.339


In [6]:
df.pnls.describe()

count   5739.000
mean       0.057
std        2.384
min      -18.554
25%        0.000
50%        0.000
75%        0.324
max       15.670
Name: pnls, dtype: float64

## Technical Indicators

### Load SPY

In [7]:
# spy_df = pd.read_csv('Data/1999-12-01-2023-12-31_SPY.csv')
# spy_df = spy_df[['Date','Adj Close']]
# spy_df.columns = ['Date','SPY_Close']
# df = pd.merge(df,spy_df,on='Date',how='left')

In [8]:
df = df.dropna()
df

,Ticker_P1,Date,High_P1,Low_P1,Volume_P1,Close_P1,Ticker_P2,High_P2,Low_P2,Volume_P2,Close_P2,abs_spread,abs_spread_mean,abs_spread_std,abs_spread_mean_MA,abs_spread_std_MA,pnls,actual_abs_spread_std
0,LHX,2001-02-08 00:00:00,13.615,13.336,589966.000,8.946,HST,12.697,12.531,289289.000,6.480,2.467,4.442,1.363,2.231,0.320,0.000,0.441
1,LHX,2001-02-09 00:00:00,13.652,12.991,774767.000,8.626,HST,12.717,12.491,368046.000,6.429,2.197,4.441,1.365,2.213,0.311,0.000,0.437
2,LHX,2001-02-12 00:00:00,13.322,12.878,939881.000,8.642,HST,12.736,12.472,436589.000,6.505,2.137,4.439,1.368,2.198,0.306,0.000,0.439
3,LHX,2001-02-13 00:00:00,13.700,13.180,930356.000,8.946,HST,12.814,12.638,435057.000,6.525,2.421,4.437,1.370,2.219,0.306,0.000,0.409
4,LHX,2001-02-14 00:00:00,13.785,13.293,881456.000,8.953,HST,12.717,12.628,560293.000,6.530,2.423,4.436,1.372,2.240,0.305,0.000,0.366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,LHX,2023-11-24 00:00:00,191.010,188.050,482300.000,191.010,HST,17.470,17.250,1743300.000,17.031,173.979,179.348,17.309,166.340,4.530,2.333,6.669
5735,LHX,2023-11-27 00:00:00,190.860,187.370,676200.000,187.920,HST,17.490,17.220,4290800.000,17.041,170.879,179.228,17.243,167.126,3.794,2.964,6.320
5736,LHX,2023-11-28 00:00:00,189.740,187.680,726600.000,189.100,HST,17.650,17.320,4449600.000,17.198,171.902,179.111,17.173,167.808,3.313,3.087,5.912
5737,LHX,2023-11-29 00:00:00,189.780,188.060,661800.000,188.660,HST,17.780,17.410,4099200.000,17.051,171.609,178.998,17.111,168.229,3.228,2.687,5.339


### Beta

In [9]:
def compute_beta(rs,df,ticker,market):
    rows = df.loc[rs.index]
    sec_returns = np.log( df[[ticker,market]] / df[[ticker,market]].shift(1) ) 

    cov = sec_returns.cov() * 250
    cov_with_market = cov.iloc[0,1]
    market_var = sec_returns[market].var() * 250
    beta = cov_with_market / market_var
    return beta

In [10]:
all_tickers = df['Ticker_P1'].unique().tolist()+df['Ticker_P2'].unique().tolist()
len(all_tickers)

2

### Single Ticker Features

In [11]:
df.columns

Index(['Ticker_P1', 'Date', 'High_P1', 'Low_P1', 'Volume_P1', 'Close_P1',
       'Ticker_P2', 'High_P2', 'Low_P2', 'Volume_P2', 'Close_P2', 'abs_spread',
       'abs_spread_mean', 'abs_spread_std', 'abs_spread_mean_MA',
       'abs_spread_std_MA', 'pnls', 'actual_abs_spread_std'],
      dtype='object')

In [12]:
df.sort_values('Date')

,Ticker_P1,Date,High_P1,Low_P1,Volume_P1,Close_P1,Ticker_P2,High_P2,Low_P2,Volume_P2,Close_P2,abs_spread,abs_spread_mean,abs_spread_std,abs_spread_mean_MA,abs_spread_std_MA,pnls,actual_abs_spread_std
0,LHX,2001-02-08 00:00:00,13.615,13.336,589966.000,8.946,HST,12.697,12.531,289289.000,6.480,2.467,4.442,1.363,2.231,0.320,0.000,0.441
1,LHX,2001-02-09 00:00:00,13.652,12.991,774767.000,8.626,HST,12.717,12.491,368046.000,6.429,2.197,4.441,1.365,2.213,0.311,0.000,0.437
2,LHX,2001-02-12 00:00:00,13.322,12.878,939881.000,8.642,HST,12.736,12.472,436589.000,6.505,2.137,4.439,1.368,2.198,0.306,0.000,0.439
3,LHX,2001-02-13 00:00:00,13.700,13.180,930356.000,8.946,HST,12.814,12.638,435057.000,6.525,2.421,4.437,1.370,2.219,0.306,0.000,0.409
4,LHX,2001-02-14 00:00:00,13.785,13.293,881456.000,8.953,HST,12.717,12.628,560293.000,6.530,2.423,4.436,1.372,2.240,0.305,0.000,0.366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,LHX,2023-11-24 00:00:00,191.010,188.050,482300.000,191.010,HST,17.470,17.250,1743300.000,17.031,173.979,179.348,17.309,166.340,4.530,2.333,6.669
5735,LHX,2023-11-27 00:00:00,190.860,187.370,676200.000,187.920,HST,17.490,17.220,4290800.000,17.041,170.879,179.228,17.243,167.126,3.794,2.964,6.320
5736,LHX,2023-11-28 00:00:00,189.740,187.680,726600.000,189.100,HST,17.650,17.320,4449600.000,17.198,171.902,179.111,17.173,167.808,3.313,3.087,5.912
5737,LHX,2023-11-29 00:00:00,189.780,188.060,661800.000,188.660,HST,17.780,17.410,4099200.000,17.051,171.609,178.998,17.111,168.229,3.228,2.687,5.339


In [13]:
df.columns

Index(['Ticker_P1', 'Date', 'High_P1', 'Low_P1', 'Volume_P1', 'Close_P1',
       'Ticker_P2', 'High_P2', 'Low_P2', 'Volume_P2', 'Close_P2', 'abs_spread',
       'abs_spread_mean', 'abs_spread_std', 'abs_spread_mean_MA',
       'abs_spread_std_MA', 'pnls', 'actual_abs_spread_std'],
      dtype='object')

In [14]:
label = 'pnls'
exclude = 'actual_abs_spread_std'
X = df
y = df[label]

X = X.drop(['Date', 'Ticker_P1', 'Ticker_P2', exclude, label],axis=1)

In [15]:
X

,High_P1,Low_P1,Volume_P1,Close_P1,High_P2,Low_P2,Volume_P2,Close_P2,abs_spread,abs_spread_mean,abs_spread_std,abs_spread_mean_MA,abs_spread_std_MA
0,13.615,13.336,589966.000,8.946,12.697,12.531,289289.000,6.480,2.467,4.442,1.363,2.231,0.320
1,13.652,12.991,774767.000,8.626,12.717,12.491,368046.000,6.429,2.197,4.441,1.365,2.213,0.311
2,13.322,12.878,939881.000,8.642,12.736,12.472,436589.000,6.505,2.137,4.439,1.368,2.198,0.306
3,13.700,13.180,930356.000,8.946,12.814,12.638,435057.000,6.525,2.421,4.437,1.370,2.219,0.306
4,13.785,13.293,881456.000,8.953,12.717,12.628,560293.000,6.530,2.423,4.436,1.372,2.240,0.305
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,191.010,188.050,482300.000,191.010,17.470,17.250,1743300.000,17.031,173.979,179.348,17.309,166.340,4.530
5735,190.860,187.370,676200.000,187.920,17.490,17.220,4290800.000,17.041,170.879,179.228,17.243,167.126,3.794
5736,189.740,187.680,726600.000,189.100,17.650,17.320,4449600.000,17.198,171.902,179.111,17.173,167.808,3.313
5737,189.780,188.060,661800.000,188.660,17.780,17.410,4099200.000,17.051,171.609,178.998,17.111,168.229,3.228


## Train test split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42,shuffle=False)

In [17]:
print(df.loc[X_val.index].Date.min(),df.loc[X_val.index].Date.max())

2019-05-13 00:00:00 2021-08-19 00:00:00


## LSTM Model

In [18]:
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Dropout
#Creating the LSTM model
class lstm(Model):
    def __init__(self, in_shape, look_back):
        super(lstm, self).__init__()
        self.shape = in_shape
        self.look_back = look_back
        self.lstm_l1 = LSTM(256, input_shape=self.shape, return_sequences = True)
#         self.lstm_l2 = LSTM(256)
        self.dropout = Dropout(0.2)
        self.dense = Dense(self.look_back)

    def call(self, x):
        x = self.lstm_l1(x)
#         x = self.lstm_l2(x)
        x = self.dropout(x)
        x = self.dense(x)
        return x

2024-03-17 22:35:36.059069: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-17 22:35:36.822723: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [19]:
# Reshape X for model training
trainX = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1])).astype(np.float32)
valX = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1])).astype(np.float32)
testX = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1])).astype(np.float32)

In [ ]:
#initializing the lstm model
import tensorflow as tf
callback = tf.keras.callbacks.EarlyStopping(monitor='val_mae', patience=5,\
                                            min_delta=0.001,restore_best_weights=True,verbose=1)

lstm_model = lstm((trainX.shape[1], trainX.shape[2]), look_back = 20)
lstm_model.compile(loss='mae', 
                   optimizer=tf.keras.optimizers.Adam(learning_rate=0.00002), metrics=['mae'])
#fitting the model
history = lstm_model.fit(
                    trainX, 
                    y_train, 
                    epochs=40, 
                    batch_size=32,
                    validation_data=(valX, y_val), 
                    verbose=1, 
                    callbacks=callback,
                    shuffle=True)

Epoch 1/40


2024-03-17 22:35:37.665070: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


144/144 [==============================] - 2s 7ms/step - loss: 1.5127 - mae: 1.5128 - val_loss: 1.6843 - val_mae: 1.6833
Epoch 2/40
144/144 [==============================] - 1s 5ms/step - loss: 1.4776 - mae: 1.4788 - val_loss: 1.6518 - val_mae: 1.6508
Epoch 3/40
144/144 [==============================] - 1s 5ms/step - loss: 1.4525 - mae: 1.4512 - val_loss: 1.6290 - val_mae: 1.6279
Epoch 4/40
144/144 [==============================] - 1s 5ms/step - loss: 1.4347 - mae: 1.4348 - val_loss: 1.6129 - val_mae: 1.6117
Epoch 5/40
144/144 [==============================] - 1s 5ms/step - loss: 1.4219 - mae: 1.4226 - val_loss: 1.6012 - val_mae: 1.6000
Epoch 6/40
144/144 [==============================] - 1s 5ms/step - loss: 1.4114 - mae: 1.4121 - val_loss: 1.5937 - val_mae: 1.5925
Epoch 7/40
144/144 [==============================] - 1s 5ms/step - loss: 1.4041 - mae: 1.4048 - val_loss: 1.5874 - val_mae: 1.5862
Epoch 8/40
144/144 [==============================] - 1s 5ms/step - loss: 1.3982 - mae:

In [ ]:
lstm_model.summary()

In [ ]:
# Plot line graph to show amount loss according the the epoch
plt.figure(figsize=(8,6))
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.title('Loss Plot -- LSTM', fontsize=14, color="black", fontweight="bold")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
inference_result = lstm_model.predict(testX).flatten()

In [ ]:
y_test.plot.hist(bins=12, alpha=0.5)
y_test.describe()

In [ ]:
pd.DataFrame(inference_result).plot.hist(bins=12, alpha=0.5)
pd.Series(inference_result).describe()

In [ ]:
lstm_mae = (inference_result[:len(y_test)] - y_test).abs().mean()
lstm_mae

In [ ]:
inference_result

In [ ]:
mean_abs_label = ((y_test).mean() - y_test).abs().mean()
# lstm_mae / y_test.abs().mean()
lstm_mae / mean_abs_label

In [ ]:
# Normality test

from scipy.stats import shapiro
# normality test
stat, p = shapiro(inference_result)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
    print('Sample looks Gaussian (fail to reject H0)')
else:
    print('Sample does not look Gaussian (reject H0)')

In [ ]:
# D’Agostino’s K^2 Test
from scipy.stats import normaltest
# normality test
stat, p = normaltest(inference_result)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
    print('Sample looks Gaussian (fail to reject H0)')
else:
    print('Sample does not look Gaussian (reject H0)')

In [ ]:
# Anderson-Darling Test
from scipy.stats import anderson
result = anderson(inference_result)
print('Statistic: %.3f' % result.statistic)
p = 0
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
if result.statistic < result.critical_values[i]:
    print('%.3f: %.3f, data looks normal (fail to reject H0)' % (sl, cv))
else:
    print('%.3f: %.3f, data does not look normal (reject H0)' % (sl, cv))